In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("news_clean/CJ_clean.csv", dtype={'ticker':str})
df

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",001040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,001040
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,001040
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,001040
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,001040
...,...,...,...,...
2433,20241219,"[리포트 브리핑]CJ, '실적 개선으로 주주환원이 확대될 자회사 가치' 목표가 16...",[서울=뉴스핌] 로보뉴스 = BNK투자증권에서 19일 CJ(001040)에 대해 '...,001040
2434,20241219,"마이크로바이옴 선두주자 '3사 3색', 누가 유리할까",이 기사는 2024년12월12일 09시05분에 팜이데일리 프리미엄 콘텐츠로 선공개 ...,001040
2435,20241223,마트 외식업 등 포인트 유효기간 연장 신세계 CJ '3년'까지,[세종=이데일리 하상렬 기자] 앞으로 대형마트 편의점 외식 등 8개 업종의 적립식 ...,001040
2436,20241225,"[현장에서]축제 분위기 넷플릭스, 진퇴양난 토종OTT",[이데일리 윤정훈 기자] 계엄령으로 차분해진 연말 도심에 핫핑크 물결이 일고 있다....,001040


In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# 1. CSV 파일 불러오기 (파일명에 맞게 수정하세요)
# ticker, date, title, body 컬럼이 있다고 가정
df = pd.read_csv("news_clean/CJ_clean.csv")

# 2. 전처리: 제목과 본문 합치기
# 제목이 핵심 내용을 담고 있으므로 앞에 배치하고, 본문을 뒤에 붙입니다.
# 결측치(NaN)가 있을 경우를 대비해 빈 문자열로 변환 후 합칩니다.
df['full_text'] = df['제목'].fillna('') + ". " + df['body'].fillna('')

# 3. ESG 분류 모델 로드 (Hugging Face Pipeline 활용)
# GPU가 있다면 device=0, CPU만 있다면 device=-1
# truncation=True: 뉴스 기사가 512 토큰(단어 조각)을 넘어가면 자동으로 뒷부분을 자릅니다.
classifier = pipeline(
    "text-classification", 
    model="yiyanghkust/finbert-esg", 
    tokenizer="yiyanghkust/finbert-esg",
    device=-1  # GPU 사용 시 0으로 변경
)

# 4. 분류 함수 정의
def analyze_esg(text):
    # 텍스트가 너무 길 경우 처리 (Pipeline이 처리하지만, 안전을 위해 길이 제한)
    # BERT 모델 최대 입력 길이는 512 토큰입니다.
    # 뉴스의 앞부분(제목+서두)에 주로 핵심이 있으므로 뒷부분이 잘려도 분류는 꽤 정확합니다.
    try:
        # truncation=True, max_length=512 옵션으로 긴 문장 처리
        result = classifier(text, truncation=True, max_length=512)[0]
        return result['label'], result['score']
    except Exception as e:
        return "Error", 0.0

# 5. 전체 데이터에 적용 (진행률 표시를 위해 tqdm 사용)
tqdm.pandas()
print("분석을 시작합니다... (데이터 양에 따라 시간이 걸릴 수 있습니다)")

# apply 함수를 통해 라벨과 점수를 추출
results = df['full_text'].progress_apply(lambda x: analyze_esg(x))

# 결과를 데이터프레임에 추가
df['esg_label'] = [res[0] for res in results]
df['esg_score'] = [res[1] for res in results]

# 6. 결과 확인 및 저장
print(df[['title', 'esg_label', 'esg_score']].head())

# E, S, G 관련 기사만 따로 보고 싶다면 필터링 가능
# esg_only_df = df[df['esg_label'] != 'None']

df.to_csv("news_data_with_esg.csv", index=False, encoding='utf-8-sig')
print("저장 완료: news_data_with_esg.csv")

KeyError: 'title'

In [9]:
df = pd.read_csv("news_clean/CJ_clean.csv", dtype={'ticker':str})
df

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",001040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,001040
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,001040
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,001040
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,001040
...,...,...,...,...
2433,20241219,"[리포트 브리핑]CJ, '실적 개선으로 주주환원이 확대될 자회사 가치' 목표가 16...",[서울=뉴스핌] 로보뉴스 = BNK투자증권에서 19일 CJ(001040)에 대해 '...,001040
2434,20241219,"마이크로바이옴 선두주자 '3사 3색', 누가 유리할까",이 기사는 2024년12월12일 09시05분에 팜이데일리 프리미엄 콘텐츠로 선공개 ...,001040
2435,20241223,마트 외식업 등 포인트 유효기간 연장 신세계 CJ '3년'까지,[세종=이데일리 하상렬 기자] 앞으로 대형마트 편의점 외식 등 8개 업종의 적립식 ...,001040
2436,20241225,"[현장에서]축제 분위기 넷플릭스, 진퇴양난 토종OTT",[이데일리 윤정훈 기자] 계엄령으로 차분해진 연말 도심에 핫핑크 물결이 일고 있다....,001040


In [12]:

import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# 1. 데이터 로드
df = pd.read_csv("news_clean/CJ_clean.csv", dtype={'ticker':str}) # 한글 기사 파일
df = df.head(5)
df['full_text'] = df['제목'].fillna('') + ". " + df['본문'].fillna('')

# 2. 다국어 지원 제로샷 모델 로드 (한국어 잘함)
# model: MoritzLaurer/mDeBERTa-v3-base-mnli-xnli (현재 다국어 제로샷 성능이 매우 좋음)
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    device=-1 # GPU 사용시 0
)

# 3. 분류할 라벨 정의 (한글로 줘도 됨)
candidate_labels = ["환경 보호 및 기후 변화", "사회적 책임 및 노동", "기업 지배구조 및 경영 투명성", "일반 뉴스"]

# 매핑용 딕셔너리 (결과를 E, S, G로 깔끔하게 저장하기 위해)
label_map = {
    "환경 보호 및 기후 변화": "Environmental",
    "사회적 책임 및 노동": "Social",
    "기업 지배구조 및 경영 투명성": "Governance",
    "일반 뉴스": "None"
}

def analyze_korean_esg(text):
    try:
        # 긴 텍스트는 앞부분 512자 정도로 자르는게 안전 (속도 및 에러 방지)
        text = text[:512] 
        
        # multi_label=False : 가장 확률 높은 하나만 뽑기
        result = classifier(text, candidate_labels, multi_label=False)
        
        # 가장 높은 점수의 라벨과 점수 가져오기
        top_label = result['labels'][0]
        top_score = result['scores'][0]
        
        return label_map[top_label], top_score
    except Exception as e:
        print(e)
        return "Error", 0.0

# 4. 실행
tqdm.pandas()
results = df['full_text'].progress_apply(analyze_korean_esg)

df['esg_label'] = [res[0] for res in results]
df['esg_score'] = [res[1] for res in results]

# 결과 확인
print(df[['title', 'esg_label', 'esg_score']].head())

# 저장
df.to_csv("korean_news_esg.csv", index=False, encoding='utf-8-sig')

Device set to use cpu
100%|██████████| 5/5 [01:27<00:00, 17.55s/it]


KeyError: "['title'] not in index"

In [13]:
df

,일자,제목,본문,ticker,full_text,esg_label,esg_score
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",001040,유통 식음료 업계 신년 화두는 `혁신과 수익창출`. [이데일리 민재용 기자] 유통 ...,None,0.433659
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,001040,[투자의맥]박스권 상단 넘어선 종목에 주목. [이데일리 경계영 기자] 대신증권은 2...,None,0.429548
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,001040,"CJ프레시웨이, 종합인증우수업체 재공인 획득. [이데일리 염지현 기자] CJ(001...",Governance,0.564147
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,001040,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다. [이데일리 이승현 ...,Social,0.344832
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,001040,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?. [이데일리 민재용 기자]...",Governance,0.339869


In [16]:
df.loc[4]['full_text']

'이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?. [이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미국에 장기 체류하며 그룹 경영 활동에 참여하지 않고 있어 그 배경에 관심이 쏠리고 있다.\n\nCJ그룹 측은 “그룹 부회장 및 그룹 비상경영위원 직위는 변동이 없다”며 이 부회장의 퇴진설을 부인했지만, 재계 일각에서는 정치권을 중심으로 사면 가능성이 제기되고 있는 동생 이재현..'

In [18]:
df = pd.read_csv('news_clean/BGF리테일_clean.csv')

In [21]:
df['본문'][7]

"[아시아경제 송화정 기자]라이프 인프라 기업 BGF리테일은 지난 21일 오후 서울 삼성동 BGF리테일 사옥에서 임직원들과 60여 개 중소협력사 관계자들이 참석한 가운데 '동반성장 특강'을 진행했다.BGF리테일은 지난해 10월 동반성장총회를 개최해 180여 중소협력사와 함께 공정거래 및 상생경영을 다짐하며 '동반성장 추진위원회'를 출범시킨 바 있다.이번에.."

In [2]:
import os

In [41]:
file_list = os.listdir('../news_clean/BGF리테일_clean.csv')

NotADirectoryError: [WinError 267] 디렉터리 이름이 올바르지 않습니다: '../news_clean/BGF리테일_clean.csv'

In [8]:
file_list

['NewsResult_20150101-20151231 (36).xlsx',
 'NewsResult_20160101-20161231 (36).xlsx',
 'NewsResult_20170101-20171231 (36).xlsx',
 'NewsResult_20180101-20181231 (37).xlsx',
 'NewsResult_20190101-20191231 (37).xlsx',
 'NewsResult_20200101-20201231 (38).xlsx',
 'NewsResult_20210101-20211231 (37).xlsx',
 'NewsResult_20220101-20221231 (36).xlsx',
 'NewsResult_20230101-20231231 (38).xlsx',
 'NewsResult_20240101-20241231 (35).xlsx']

In [10]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

for file in file_list:
    df2 = pd.read_excel(f'../news_clean/BGF리테일_clean/{file}')
    df = pd.concat([df, df2], ignore_index=True)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply o

In [19]:
df = df[['일자', '제목', '키워드', '본문']]

In [24]:
df.rename(columns={'키워드':'keywords', '제목':'title'}, inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_11864\1008964455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'키워드':'keywords', '제목':'title'}, inplace=True)


In [25]:
import pandas as pd
from tqdm import tqdm

# 1. 데이터 로드 (빅카인즈 데이터 예시)
# 실제 파일명으로 변경해주세요. 컬럼명은 보통 '키워드' 혹은 'keywords' 입니다.
# df = pd.read_csv("news_data_146m.csv")

# 데이터가 없는 경우를 위해 결측치 처리
df['keywords'] = df['keywords'].fillna("")

# ---------------------------------------------------------
# 2. ESG 키워드 세트 정의 (Set 자료구조 사용 -> 속도 최적화)
# ---------------------------------------------------------
# 앞서 정의한 키워드들을 set으로 변환하여 검색 속도를 O(1)로 만듭니다.

set_e = set([
    "탄소", "온실가스", "기후", "친환경", "재생에너지", "태양광", "풍력", "수소", 
    "폐기물", "재활용", "리사이클", "오염", "미세먼지", "정화", "녹색", "저탄소", 
    "배출권", "에너지효율", "넷제로", "RE100", "생태계", "환경보호", "전기차", "2차전지"
])

set_s = set([
    "노동", "인권", "고용", "일자리", "노사", "노조", "파업", "임금", "복지", 
    "안전", "중대재해", "산재", "개인정보", "정보보호", "소비자", "상생", 
    "동반성장", "사회공헌", "기부", "후원", "양성평등", "다양성", "갑질", "하도급"
])

set_g = set([
    "지배구조", "이사회", "사외이사", "주주", "의결권", "배당", "자사주", 
    "경영권", "승계", "감사", "투명성", "윤리", "부패", "횡령", "배임", 
    "뇌물", "공정거래", "일감몰아주기", "지주사", "불공정"
])

# ---------------------------------------------------------
# 3. 분류 함수 정의
# ---------------------------------------------------------
def classify_esg_by_keywords(keyword_string):
    if not keyword_string:
        return "None", 0

    # 빅카인즈 키워드는 보통 "단어1,단어2,단어3" 형태임
    # 콤마로 쪼개고 공백 제거
    article_keywords = set([k.strip() for k in keyword_string.split(",")])
    
    # 교집합 개수 확인 (매칭된 단어 개수)
    count_e = len(article_keywords & set_e)
    count_s = len(article_keywords & set_s)
    count_g = len(article_keywords & set_g)
    
    # 가장 많이 매칭된 카테고리 선정
    counts = {"Environmental": count_e, "Social": count_s, "Governance": count_g}
    
    # 매칭된 키워드가 하나도 없으면 None
    if sum(counts.values()) == 0:
        return "None", 0
    
    # 가장 높은 점수의 키워드 찾기 (동점일 경우 E->S->G 순 우선순위, 혹은 먼저 나온 것)
    best_category = max(counts, key=counts.get)
    score = counts[best_category] # 매칭된 키워드 개수를 '점수'로 활용 가능
    
    return best_category, score

# ---------------------------------------------------------
# 4. 전체 데이터 적용
# ---------------------------------------------------------
tqdm.pandas()
print("빅카인즈 키워드로 분류 시작...")

# apply 함수로 적용
results = df['keywords'].progress_apply(classify_esg_by_keywords)

# 결과 컬럼 생성
df['esg_label'] = [res[0] for res in results]
df['match_count'] = [res[1] for res in results] # 신뢰도 지표로 사용 가능

# ---------------------------------------------------------
# 5. 결과 확인 및 저장
# ---------------------------------------------------------
# 분류된 것만 보기
filtered_df = df[df['esg_label'] != "None"]

print(filtered_df[['title', 'keywords', 'esg_label', 'match_count']].head())
print(f"분류된 기사 수: {len(filtered_df)}건")

# 저장
# df.to_csv("news_esg_classified_by_keywords.csv", index=False, encoding="utf-8-sig")

C:\Users\student\AppData\Local\Temp\ipykernel_11864\189217885.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].fillna("")


빅카인즈 키워드로 분류 시작...


100%|██████████| 35489/35489 [00:01<00:00, 20951.66it/s]

                                    title  \
0           삼성전자 현대重 CJ E&M..저평가 中 테마주 주목   
2  콜라보레이션 한정판, 식탁에도 CJ알래스카연어 라인프렌즈 브라운에디션   
4     삼성전자 투자 1순위 LG화학 CJ E&M 기아차도 '반짝반짝'   
7                    CJ, 베트남 학생들에게 장학금 지원   
8       삼성증권, "새해 CJ CGV 중국 관련 엔터 주식에 주목"   

                                            keywords      esg_label  \
0  삼성전자,현대,CJ,E&M..저,평가,테마주,주목,증권사,톱픽,병신년,丙申年,새해,...  Environmental   
2  콜라보레이션,한정판,식탁,브라운,CJ,알래스카연어,라인프렌즈,디션,캐릭터,문화,트렌...         Social   
4  1순위,삼성전자,투자,LG화학,2016년,증권사,추천,유망주,인상,미국,금리,이탈,...     Governance   
7  CJ,베트남,학생,장학금,지원,CJ그룹,베트남,람동,대학생,55명,장학금,지원,CJ...         Social   
8  삼성증권,주식,새해,CJ,CGV,중국,엔터,주목,삼성증권,2016년,중국,관련주,주...  Environmental   

   match_count  
0            1  
2            1  
4            2  
7            1  
8            2  
분류된 기사 수: 11926건



C:\Users\student\AppData\Local\Temp\ipykernel_11864\189217885.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['esg_label'] = [res[0] for res in results]
C:\Users\student\AppData\Local\Temp\ipykernel_11864\189217885.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['match_count'] = [res[1] for res in results] # 신뢰도 지표로 사용 가능


In [28]:
df

,일자,title,keywords,본문,esg_label,match_count
0,20151231,삼성전자 현대重 CJ E&M..저평가 中 테마주 주목,"삼성전자,현대,CJ,E&M..저,평가,테마주,주목,증권사,톱픽,병신년,丙申年,새해,...",증권사가 꼽은 2016년 톱픽 \n \n \n 병신년(丙申年) 새해가 밝았지만 여전...,Environmental,1
1,20151231,[실시간 채용정보] 대한전선ㆍCJ올리브네트웍스ㆍ신세계아이앤씨 등,"대한전선,CJ올리브네트웍스ㆍ신세계아이앤씨,대한전선,신입,상반기,경력사원,채용,모집부...",[이투데이] 기정아 기자(jjonga1006@etoday.co.kr)대한전선에서 2...,None,0
2,20151231,"콜라보레이션 한정판, 식탁에도 CJ알래스카연어 라인프렌즈 브라운에디션","콜라보레이션,한정판,식탁,브라운,CJ,알래스카연어,라인프렌즈,디션,캐릭터,문화,트렌...",[이투데이] 이선애 기자(lsa@etoday.co.kr)최근 캐릭터 문화가 트렌드로...,Social,1
3,20151231,"“교육 불평등으로 가난 대물림 안돼” 이재현 CJ 회장의 철학, 베트남서 전파","교육,가난,대물림,회장,이재현,CJ,철학,베트남,전파,CJ그룹,베트남,람동,대학생,...",[이투데이] 이선애 기자(lsa@etoday.co.kr)CJ그룹이 베트남 람동성의 ...,None,0
4,20151231,삼성전자 투자 1순위 LG화학 CJ E&M 기아차도 '반짝반짝',"1순위,삼성전자,투자,LG화학,2016년,증권사,추천,유망주,인상,미국,금리,이탈,...",미국 금리 인상에 따른 신흥국 자금 이탈과 중국 경기 둔화 우려까지 겹쳐 있어 새해...,Governance,2
...,...,...,...,...,...,...
35484,20240102,"[신년사] 손경식 CJ 회장, ""구조적 변화 가속화 온리원 정신 재건""","회장,손경식,CJ,가속화,구조,변화,가속,온리원,정신,재건,초격차,역량,수익,극대,...",손경식(사진) CJ(001040)그룹 회장은 2일 신년사를 통해 “올해 경영환경은 ...,None,0
35485,20240102,"［신년사］ 손경식 CJ 회장, “ONLYONE 정신 회복해야”","신년사,회장,손경식,CJ,회복,ONLYONE,정신,손경식,CJ,그룹,회장,CJ,브레...",▲ 손경식 CJ그룹 회장 ©CJ \n \n\n\n\n\n\n\n\n\n\n\...,None,0
35486,20240102,"[신년사] CJ 손경식 회장 ""현실 안주 말고 온리원 정신 회복하자""","안주,CJ,손경식,회장,현실,회복,온리원,정신,아시아투데이,김지혜,회장,손경식,CJ...",아시아투데이 김지혜 기자 = 손경식 CJ그룹 회장이 2일 사내방송을 통해 '2024...,None,0
35487,20240102,손경식 CJ 회장 “온리원 정신으로 초격차 1등 달성” [신년사],"정신,손경식,CJ,회장,온리원,달성,초격차,신년사,회장,손경식,CJ,그룹,새해,그룹...",[이투데이] 김지영 기자 (kjy42@etoday.co.kr)\n\n\n\n손경식 ...,None,0


In [32]:
df.loc[4]

일자                                                      20151231
title                        삼성전자 투자 1순위 LG화학 CJ E&M 기아차도 '반짝반짝'
keywords       1순위,삼성전자,투자,LG화학,2016년,증권사,추천,유망주,인상,미국,금리,이탈,...
본문             미국 금리 인상에 따른 신흥국 자금 이탈과 중국 경기 둔화 우려까지 겹쳐 있어 새해...
esg_label                                             Governance
match_count                                                    2
Name: 4, dtype: object

In [33]:
df.loc[4]['keywords']

'1순위,삼성전자,투자,LG화학,2016년,증권사,추천,유망주,인상,미국,금리,이탈,신흥국,자금,둔화,중국,경기,새해,증시,새해,상승세,증권가,전망,상황,지수,추종,위주,특정,업종,포트폴리오,여부,실적,개선,성장,가능,종목,투자,유리,서울경제신문,31일,증권사,유망,종목,추천,결과,삼성전자,LG화학,E&M,종목,추천,종목,소개,삼성전자,제품경쟁력,사업다각화,2분기,예상,실적,개선,삼성전자,증권사,유망종목,추천,둔화,스마트폰,수요,메모리,수요,중국,진출,메모리,산업,위협,요인,제품,경쟁력,사업,다각,안정적,이익,예상,증권사들,추정,연간,영업,이익,규모,27조,안팎,1,2,실적,대체,증권가,전망,삼성전자,유망종목,주주,환원,확대,정책,배당금액,연간,지속,자사,매입,소각,주가,대신증권,가용,현금,흐름,안정,국면,진입,환원,주주,이익,확대,평가,LG화학,전기차,시장,확대,수혜,배터리사업,영업익,확대,전기차,시장,수혜주,4분기,분기,전기차,배터리,실적,개선,자동차,배터리,포함,배터리,사업,영업이익,기록,전망,전체적,21조,원대,중반,매출,2조,000억,원대,영업이익,달성,증권가,예상,현대증권,중국,전기차,시장,확대,LG화학,최대,수혜,감소,모바일,전지,매출,모델,재고,폐기,지적,CJ,E&M,영화,시장,급성장,호재,매출액,000억,전망,폭발적,시장,중국,영화,CJ,E&M,주가,호재,작용,전망,규모,중국,박스오피스,2015년,50%,확대,폭발적,성장세,합작영화들,흥행,여부,관심,평가,증권,정보,업체,에프앤가이드,CJ,E&M,매출액,연간,1조,143억,영업이익,881억,전망,연구원,임민규,현대증권,방송,호조세,케이블,채널,사업,지속,중심,중국,방송,프로그램,수익원,확대,전망,영화,합작,영화,3편,확대,외형,성장,지속,동아에스티,확대,항생제,글로벌,판매,경상,기술료,수입,신약,항생제,시벡스트,확대,글로벌,판매,경상,기술료,수입,임상시험,진행,바이오시밀러,복제약,천연물,새해,주가,상승,모멘텀,해외사업부,고성장,주목,투자,포인트,연구원,김주용,키움증권,박카스,확대,글로벌,유통,채

In [31]:
df.loc[0]['본문']

'증권사가 꼽은 2016년 톱픽 \n \n \n 병신년(丙申年) 새해가 밝았지만 여전히 전 세계 경기는 저성장.저물가 기조 속에 안갯속이다. 전문가들은 실적 개선이 예상되는 저평가 대형주를 중심으로 압축적인 대응을 권하고 있다. 동시에 중국의 소피팽창 전략에 대응해 엔터, 전기차, 제약 등은 불확실성을 뛰어넘어 급성장 할 것으로 내다봤다. \n \n■실적개선 대형..'

In [36]:
env = df.loc[df['esg_label'] == 'Environmental']

In [37]:
env

,일자,title,keywords,본문,esg_label,match_count
0,20151231,삼성전자 현대重 CJ E&M..저평가 中 테마주 주목,"삼성전자,현대,CJ,E&M..저,평가,테마주,주목,증권사,톱픽,병신년,丙申年,새해,...",증권사가 꼽은 2016년 톱픽 \n \n \n 병신년(丙申年) 새해가 밝았지만 여전...,Environmental,1
8,20151230,"삼성증권, ""새해 CJ CGV 중국 관련 엔터 주식에 주목""","삼성증권,주식,새해,CJ,CGV,중국,엔터,주목,삼성증권,2016년,중국,관련주,주...",삼성증권이 2016년에도 중국 관련주에 주목할 것을 조언했다.\n \n새해에도 중국...,Environmental,2
206,20151212,[주간추천종목] 신한금융투자(한화케미칼 CJ CGV LG화학),"신한금융투자,한화케미칼,CJ,CGV,한화케미칼,개선,제품,믹스,출하량,증가,하락,제...","◇ 한화케미칼 - 제품 믹스 개선, 출하량 증가, 제조 원가 하락으로 태양광 부문 ...",Environmental,2
233,20151208,"‘CJ헬로 인수’ 장동현 SKT 사장의 한마디 ""각자 갈길 다르다""","CJ헬,인수,사장,장동현,SKT,한마디,통신3사,분야,집중,경쟁사,합병,결과적,미래...",“우리도 경쟁사 합병 시 우려를 표한 바 있다. 그런데 돌아보면 결과적으로 미래를 ...,Environmental,1
238,20151208,"장동현 SK텔레콤 사장 ""CJ헬로비전 인수합병으로 시장 변화 선도""","인수합병,장동현,SK,텔레콤,사장,CJ,헬로비전,인수,합병,선도,시장,변화,반복,싸...","""싸이월드 아픔 반복하더라도 변화와 진화 앞장설 것"" \n \n장동현 SK텔레콤 사...",Environmental,1
...,...,...,...,...,...,...
35219,20240207,“CJ올리브영에서만 1000억 매출” 중소 K뷰티 브랜드 뜬다,"CJ올리브영,매출,중소,뜬다,브랜드,업체들,중소,화장품,업체,수출,역군,생태계,산업...",중소 화장품 업체들이 새로운 수출 역군으로 떠오르며 이들을 키운 K뷰티 산업 생태계...,Environmental,1
35398,20240112,'제2의 코엑스' 꿈꾸는 가양동 CJ부지 PF가 관건,"제2,코엑스,가양동,CJ,부지,PF,스타필드빌리지,2,소송전,구청장,교체,사업,속도...",서울 강서구 가양동에 '제2의 코엑스'를 표방한 복합 상업시설이 추진되고 있다. 총...,Environmental,1
35412,20240111,[기업] CJ 이재현 회장 새해 첫 현장경영 '올리브영' 본사 방문,"CJ,이재현,회장,새해,현장,경영,올리브영,방문,회장,이재현,CJ,그룹,코로나19,...",이재현 CJ그룹 회장이 코로나19 이후 처음으로 계열사를 방문해 협력업체와의 상생을...,Environmental,1
35433,20240109,"기아, PBV 전략 공개 “우버 쿠팡 CJ대한통운과도 협업” [CES 2024]","기아,PBV,전략,공개,우버,쿠팡,CJ대한통운,협업,CES,목적기반모빌리티,Purp...","“목적기반모빌리티(Purpose-Bulit Vehicle)를 넘어, 차량 그 이상의...",Environmental,1
